# Reddit Depression Final Project
Link to the paper: https://dl.acm.org/doi/pdf/10.1145/3578503.3583621

Read through the paper fully before starting the assignment!

In [ ]:
!pip install dlatk
!pip install happiestfuntokenizing
!pip install gensim
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.1/88.1 kB 11.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 83.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 94.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 59.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.7/233.7 kB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.4/30.4 MB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 MB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 93.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 35.0 MB/s eta 0:00:00
  Created wheel for dlatk: filename=dlatk-1.2.7-py3-none-any.whl size=20684633 sha256=1e27d442ff20804d8dba8d1f5

  Preparing metadata (setup.py) ... done
  Created wheel for happiestfuntokenizing: filename=happiestfuntokenizing-0.0.7-py3-none-any.whl size=6710 sha256=cdc1342dbe31f555d22744e492348d4cebfe3a1575d6e0a08a73064b9c78c6ae
  Stored in directory: /root/.cache/pip/wheels/bf/c9/4d/310f0c60855eb7b428558f29d93cf464dbb64c1b8628753395
Successfully built happiestfuntokenizing
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 70.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.9 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_validate, cross_val_score, KFold
from sklearn.ensemble import RandomForestClassifier
import pickle
from collections import Counter
import gensim
import gensim.corpora as corpora
from transformers import RobertaTokenizer, RobertaModel
import torch
from gensim.models.ldamulticore import LdaMulticore
from happiestfuntokenizing.happiestfuntokenizing import Tokenizer
from sklearn.metrics import roc_auc_score
from collections import Counter


from google.colab import drive
drive.mount('/content/drive')

FILEPATH = 'drive/MyDrive/1460HW/student.pkl'

Mounted at /content/drive


## Preprocessing

In [ ]:
def load(file_path):
    """
    Load and return the data from a pickle file.

    Parameters:
    file_path (str): The path to the .pkl file to be loaded.

    Returns:
    data: The data loaded from the .pkl file.
    """
    with open(file_path, 'rb') as file:
        data = pickle.load(file)
    return data

# call the function to display the dataset
data = load(FILEPATH)
data.head()


,text,author,subreddit,created_utc,date
0,does your life feel like a waste mines not a c...,trademeple,depression,1504920055,2017-09
1,Just relapsed again. Any advice I just got to ...,kenny818,NoFap,1507890053,2017-10
2,Audio and mic not working? So I have a HyperX ...,psyjinks,techsupport,1513558467,2017-12
3,PG&amp;E: Mylar balloon causes outage in centr...,Majnum,nottheonion,1499573023,2017-07
4,Um... Forward?,OldManoftheNorth,memes,1516842851,2018-01


In [ ]:
# List of depression subreddits in the paper
depression_subreddits = ["Anger",
    "anhedonia", "DeadBedrooms",
    "Anxiety", "AnxietyDepression", "HealthAnxiety", "PanicAttack",
    "DecisionMaking", "shouldi",
    "bingeeating", "BingeEatingDisorder", "EatingDisorders", "eating_disorders", "EDAnonymous",
    "chronicfatigue", "Fatigue",
    "ForeverAlone", "lonely",
    "cry", "grief", "sad", "Sadness",
    "AvPD", "SelfHate", "selfhelp", "socialanxiety", "whatsbotheringyou",
    "insomnia", "sleep",
    "cfs", "ChronicPain", "Constipation", "EssentialTremor", "headaches", "ibs", "tinnitus",
    "AdultSelfHarm", "selfharm", "SuicideWatch",
    "Guilt", "Pessimism", "selfhelp", "whatsbotheringyou"
]

In [ ]:
# Map the subreddits to each symptom
subreddit_to_sympton = {
    "Anger": "Anger",
    "anhedonia": "Anhedonia",
    "DeadBedrooms": "Anhedonia",
    "Anxiety": "Anxiety",
    "AnxietyDepression": "Anxiety",
    "HealthAnxiety": "Anxiety",
    "PanicAttack": "Anxiety",
    "DecisionMaking": "Concentration deficit",
    "shouldi": "Concentration deficit",
    "bingeeating": "Disordered eating",
    "BingeEatingDisorder": "Disordered eating",
    "EatingDisorders": "Disordered eating",
    "eating_disorders": "Disordered eating",
    "EDAnonymous": "Disordered eating",
    "chronicfatigue": "Fatigue",
    "Fatigue": "Fatigue",
    "ForeverAlone": "Loneliness",
    "lonely": "Loneliness",
    "cry": "Sad mood",
    "grief": "Sad mood",
    "sad": "Sad mood",
    "Sadness": "Sad mood",
    "AvPD": "Self-loathing",
    "SelfHate": "Self-loathing",
    "selfhelp": "Self-loathing",
    "selfhelp": "Worthlessness",
    "socialanxiety": "Self-loathing",
    "whatsbotheringyou": "Self-loathing",
    "whatsbotheringyou": "Worthlessness",
    "insomnia": "Sleep problem",
    "sleep": "Sleep problem",
    "cfs": "Somatic complaint",
    "ChronicPain": "Somatic complaint",
    "Constipation": "Somatic complaint",
    "EssentialTremor": "Somatic complaint",
    "headaches": "Somatic complaint",
    "ibs": "Somatic complaint",
    "tinnitus": "Somatic complaint",
    "AdultSelfHarm": "Suicidal thoughts and attempts",
    "selfharm": "Suicidal thoughts and attempts",
    "SuicideWatch": "Suicidal thoughts and attempts",
    "Guilt": "Worthlessness",
    "Pessimism": "Worthlessness",
}


# Custom function to get symptom
def get_symptom(subreddit):
    return subreddit_to_sympton.get(subreddit, 'Control')

# Use apply to create the new column 'symptom'
data['symptom'] = data['subreddit'].apply(get_symptom)



In [ ]:
def dataset_generation(data, depression_subs):
    """
    Build control and symptom datasets.

    Parameters:
    - data: pandas DataFrame containing the Reddit posts data
    - depression_subs: list of subreddits related to depression symptoms

    Returns:
    - symptom_df: DataFrame containing posts from depression-related subreddits
    - control_df: DataFrame containing control posts that are at least 180 days older than the author's first depression-related post
    """
    # Filter out symptom dataset where symptom not equal to control
    symptom_df = data[data['symptom']!= 'Control']

     # Convert 'created_utc' to datetime
    data['date'] = pd.to_datetime(data['created_utc'], unit='s')

    # Identify authors with depression-related posts and find the earliest post's created_utc
    depression_authors = symptom_df.groupby('author')['created_utc'].min()

    # Map the minimum created_utc of depression posts for each author back to the original dataframe
    data['min_depression_utc'] = data['author'].map(depression_authors)

    # Calculate the time difference in days between each post and the earliest depression post for the same author
    data['days_diff'] = (data['created_utc'] - data['min_depression_utc']) / (60 * 60 * 24)

    # Filter for control posts that are at least 180 days older than the author's earliest depression-related post
    control_df = data[(data['symptom'] == 'Control') & (data['days_diff'] <= -180)]


    # Return the symptom and control dataframes
    return symptom_df, control_df


symptom_data, control_data = dataset_generation(data, depression_subreddits)



In [ ]:
# Check the shape of symptom and control dataset
print('sympton dataset shape:', symptom_data.shape, 'control dataset shape:', control_data.shape)

sympton dataset shape: (94514, 6) control dataset shape: (4369, 8)


In [ ]:
# Tokenize the text data

def tokenize(data):
  """Tokenize each message in the DataFrame."""

  tokenizer = Tokenizer(preserve_case=False)

  # Apply tokenization to each message
  data['tokens'] = data['text'].apply(tokenizer.tokenize)

  return data

In [ ]:
def stop_words(data, n=100):
    """
    Find top n words from the dataset to use as stop words.

    Parameters:
    - data: pandas DataFrame containing the dataset.
    - n: Number of top words to return. Default is 100.

    Returns:
    - top_n_words: List containing the top n words.
    """

    # Flatten the list of lists of tokens into a single list
    all_tokens = [token for sublist in data['tokens'].tolist() for token in sublist]

    # Create a frequency distribution of the tokens
    word_freq = Counter(all_tokens)

    # Select the top n words
    top_n_words = [word for word, freq in word_freq.most_common(n)]

    return top_n_words

##Dataframe for LDA and RoBERTa features, symptom

In [ ]:
# Combine control_data and symptom_data
control_data = control_data[['symptom', 'text']]
symptom_data = symptom_data[['symptom', 'text']]


# Now concatenate them
combined_data = pd.concat([symptom_data, control_data], ignore_index=True)


# Initialize an empty DataFrame to store features
features_df = pd.DataFrame()

# Add the symptoms and text columns from your original data to this new DataFrame
features_df['symptom'] = combined_data['symptom']
features_df['text'] = combined_data['text']
features_df['label'] = None


# You can add empty columns for LDA and RoBERTa features which you will fill later
features_df['lda_features'] = None
features_df['roberta_features'] = None


In [ ]:
features_df.head()

,symptom,text,label,lda_features,roberta_features
0,Suicidal thoughts and attempts,"i'm trying hi, i'm sorry if my writing is bad,...",None,None,None
1,Loneliness,Only friend has been blanking me for what feel...,None,None,None
2,Anxiety,Study hall social anxiety bruh We had a study ...,None,None,None
3,Anxiety,Positive Thoughts For You - We Are Happy To Pu...,None,None,None
4,Suicidal thoughts and attempts,Starting from a blowup mattress Today was a ve...,None,None,None


## Reddit Topics with LDA

 - Don't use MALLET (as the paper does), use some other LDA implementation.

In [ ]:
# TODO: Your LDA code!
# Function to tokenize and remove stop words

def preprocess_data(tokenize_data):
    # Tokenize the data
    tokenize_data = tokenize(tokenize_data)
    #Get the top 100 words
    top_100_stop_words = stop_words(tokenize_data, n=100)
    # Remove the top 100 words
    tokenize_data['remove_stop_word_tokens'] = tokenize_data['tokens'].apply(lambda tokens: [token for token in tokens if token not in top_100_stop_words])

    return tokenize_data

# Function to implement LDA feature
def lda_for_each_symptom(features_df):
    lda_models = {}

    # Convert symptoms into a list
    symptoms = features_df['symptom'].unique().tolist()

    # Preprocess data by tokenize and removing stop words
    features_df = preprocess_data(features_df)

    # Run LDA for each symptom
    for symptom in symptoms:
        current_data = features_df[features_df['symptom'] == symptom]
        if current_data.empty:
            print(f"No data available for symptom '{symptom}'.")
            continue

        # Create a dictionary and corpus for the current symptom
        id2word_current = corpora.Dictionary(current_data['remove_stop_word_tokens'])

        # id2word_current.filter_extremes(no_below=5, no_above=0.5, keep_n=100000)
        if len(id2word_current) == 0:
            print(f"No words left after filtering for symptom '{symptom}'.")
            continue

        corpus_current = [id2word_current.doc2bow(text) for text in current_data['remove_stop_word_tokens']]
        if not corpus_current:
            print(f"No corpus could be built for symptom '{symptom}'.")
            continue

        # Create LDA model for the current symptom
        lda_model_current = LdaMulticore(corpus=corpus_current,
                                         id2word=id2word_current,
                                         num_topics=200,
                                         passes=5
                                        )


        # Populate the 'lda_features' column for the corresponding symptom
        for index, row in features_df[features_df['symptom'] == symptom].iterrows():
            bow = id2word_current.doc2bow(row['remove_stop_word_tokens'])
            features_df.at[index, 'lda_features'] = lda_model_current.get_document_topics(bow)

    return features_df

features_df = lda_for_each_symptom(features_df)


## RoBERTa Embeddings

In [ ]:
import torch
from transformers import RobertaTokenizer, RobertaModel
from torch.utils.data import DataLoader, TensorDataset

# Clear CUDA memory to free up unused memory and avoid potential out-of-memory errors.
torch.cuda.empty_cache()

# Set the device for computation. If a CUDA-compatible GPU is available, use it; otherwise, default to CPU.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize the RoBERTa tokenizer and model. The RoBERTa model is a pre-trained contextual word embedding model,
# which generates numerical vector representations of language. The 'roberta-base' model is used here.
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaModel.from_pretrained('roberta-base', output_hidden_states=True)

# Move the model to the specified device (GPU).
model.to(device)

# Set the model to evaluation mode. This deactivates dropout layers and prepares the model for inference.
model.eval()

# Function to create a DataLoader for processing texts in batches.
def create_data_loader(texts, batch_size):
    # Tokenize the input texts. This process converts text data into a format that can be processed by RoBERTa.
    # Padding and truncation ensure consistent sequence length. 'max_length' limits the token length.
    encoded_input = tokenizer(texts, padding=True, truncation=True, return_tensors='pt', max_length=512)

    # Create a TensorDataset from the tokenized input, which includes input_ids and attention masks.
    # The DataLoader will use this dataset to generate batches of data.
    dataset = TensorDataset(encoded_input['input_ids'], encoded_input['attention_mask'])
    data_loader = DataLoader(dataset, batch_size=batch_size)
    return data_loader

# Function to obtain RoBERTa embeddings for each text using the DataLoader for batch processing.
def get_roberta_embeddings(data_loader):
    roberta_features_list = []
    with torch.no_grad():  # Disable gradient calculations for efficiency.
        for batch in data_loader:
            # Load each batch to the specified device GPU.
            input_ids, attention_mask = [b.to(device) for b in batch]

            # Get model outputs, which include hidden states. The hidden states are layers' output embeddings.
            outputs = model(input_ids, attention_mask=attention_mask)
            layer_embeddings = outputs.hidden_states[10]  # Extract embeddings from the 10th layer.

            # For each text in the batch, calculate the mean of all token embeddings to represent the text.
            batch_features = [torch.mean(embedding, dim=0).cpu().numpy().tolist() for embedding in layer_embeddings]
            roberta_features_list.extend(batch_features)
    return roberta_features_list

# Extract the texts from the DataFrame.
texts = features_df['text'].tolist()

# Define the batch size for the DataLoader.
batch_size = 32  # Adjust as needed based on available memory.

# Create a DataLoader with the texts and specified batch size.
data_loader = create_data_loader(texts, batch_size)

# Obtain RoBERTa embeddings for the texts and store them in the DataFrame.
features_df['roberta_features'] = get_roberta_embeddings(data_loader)


(…).co/roberta-base/resolve/main/vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

(…).co/roberta-base/resolve/main/merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

(…)roberta-base/resolve/main/tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

(…)co/roberta-base/resolve/main/config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
features_df.head()

,symptom,text,label,lda_features,roberta_features,tokens,remove_stop_word_tokens
0,Suicidal thoughts and attempts,"i'm trying hi, i'm sorry if my writing is bad,...",None,"[(2, 0.24948725), (12, 0.010981856), (13, 0.04...","[0.2426946610212326, -0.15012778341770172, 0.1...","[i'm, trying, hi, ,, i'm, sorry, if, my, writi...","[trying, hi, sorry, writing, bad, headache, di..."
1,Loneliness,Only friend has been blanking me for what feel...,None,"[(8, 0.6873028), (22, 0.022346536), (66, 0.020...","[-0.19960957765579224, 0.10301163792610168, 0....","[only, friend, has, been, blanking, me, for, w...","[friend, blanking, feels, months, complete, sh..."
2,Anxiety,Study hall social anxiety bruh We had a study ...,None,"[(11, 0.013373465), (14, 0.013308721), (21, 0....","[0.10037973523139954, -0.48160794377326965, 0....","[study, hall, social, anxiety, bruh, we, had, ...","[study, hall, social, bruh, study, hall, gym, ..."
3,Anxiety,Positive Thoughts For You - We Are Happy To Pu...,None,"[(9, 0.2417878), (21, 0.36158147), (158, 0.199...","[0.02830909565091133, 0.13075783848762512, -0....","[positive, thoughts, for, you, -, we, are, hap...","[positive, thoughts, happy, publish, !]"
4,Suicidal thoughts and attempts,Starting from a blowup mattress Today was a ve...,None,"[(25, 0.03494058), (26, 0.09461491), (32, 0.07...","[0.08719241619110107, -0.2380009889602661, 0.2...","[starting, from, a, blowup, mattress, today, w...","[starting, blowup, mattress, today, very, “, d..."


In [ ]:
features_df.to_csv('features_df,csv', index = False)

## Main

In [ ]:
# Get the ten required symptoms
symptom_list = ['Loneliness', 'Anxiety',
       'Anhedonia', 'Sleep problem', 'Self-loathing', 'Sad mood',
       'Somatic complaint', 'Disordered eating', 'Worthlessness', 'Anger']

In [ ]:
def main(features_df):
    # Initialize a RandomForestClassifier
    rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

    # Initialize a K-Fold cross-validator
    cv = KFold(n_splits=5, shuffle=True, random_state=42)

    # Create an empty DataFrame to store the results
    scores_df = pd.DataFrame(columns=['Symptom', 'LDA Score', 'RoBERTa Score'])

    # Iterate over each unique symptom in the dataset.
    for symptom in symptom_list:
        # Filter the DataFrame for the current symptom and control group, creating a subset DataFrame.
        symptom_df = features_df[(features_df['symptom'] == symptom) | (features_df['symptom'] == 'Control')].copy()

        # Create a new column 'label' in the subset DataFrame, which is 1 for the current symptom and 0 for control.
        symptom_df['label'] = (symptom_df['symptom'] == symptom).astype(int)

        # Convert LDA features to a uniform length.
        #Flatten the features to one-dimensional, find the maximum length and pad shorter arrays to this length.
        X_lda = np.array([np.array(f).flatten() for f in symptom_df['lda_features']])
        lda_max_len = max(len(f) for f in X_lda)
        X_lda = np.array([np.pad(f, (0, lda_max_len - len(f)), 'constant') for f in X_lda])

        # Repeat the same process for RoBERTa features.
        X_roberta = np.array([np.array(f).flatten() for f in symptom_df['roberta_features']])
        roberta_max_len = max(len(f) for f in X_roberta)
        X_roberta = np.array([np.pad(f, (0, roberta_max_len - len(f)), 'constant') for f in X_roberta])

        # Extract the labels for the current subset.
        y = symptom_df['label'].values

        # Perform cross-validation for LDA features and calculate the mean score across all folds.
        lda_scores = cross_val_score(rf_classifier, X_lda, y, cv=cv, scoring='roc_auc')
        lda_mean_score = lda_scores.mean()

        # Repeat the cross-validation for RoBERTa features and calculate the mean score.
        roberta_scores = cross_val_score(rf_classifier, X_roberta, y, cv=cv, scoring='roc_auc')
        roberta_mean_score = roberta_scores.mean()

        # Append the mean scores along with the symptom to the result DataFrame.
        scores_df = scores_df.append({'Symptom': symptom, 'LDA Score': lda_mean_score, 'RoBERTa Score': roberta_mean_score}, ignore_index=True)

    return scores_df

# Call the main function with the DataFrame containing features.
result = main(features_df)

# Display the result DataFrame.
result

<ipython-input-19-b901288d7536>:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_lda = np.array([np.array(f).flatten() for f in symptom_df['lda_features']])
<ipython-input-19-b901288d7536>:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  scores_df = scores_df.append({'Symptom': symptom, 'LDA Score': lda_mean_score, 'RoBERTa Score': roberta_mean_score}, ignore_index=True)
<ipython-input-19-b901288d7536>:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_lda =

             Symptom  LDA Score  RoBERTa Score
0         Loneliness   0.857270       0.886347
1            Anxiety   0.904333       0.933635
2          Anhedonia   0.889810       0.935636
3      Sleep problem   0.843853       0.926923
4      Self-loathing   0.871455       0.915678
5           Sad mood   0.773589       0.905466
6  Somatic complaint   0.837220       0.904774
7  Disordered eating   0.922586       0.922055
8      Worthlessness   0.847880       0.883317
9              Anger   0.919084       0.906059


<ipython-input-19-b901288d7536>:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  scores_df = scores_df.append({'Symptom': symptom, 'LDA Score': lda_mean_score, 'RoBERTa Score': roberta_mean_score}, ignore_index=True)


In [ ]:
result

,Symptom,LDA Score,RoBERTa Score
0,Loneliness,0.857270,0.886347
1,Anxiety,0.904333,0.933635
2,Anhedonia,0.889810,0.935636
3,Sleep problem,0.843853,0.926923
4,Self-loathing,0.871455,0.915678
5,Sad mood,0.773589,0.905466
6,Somatic complaint,0.837220,0.904774
7,Disordered eating,0.922586,0.922055
8,Worthlessness,0.847880,0.883317
9,Anger,0.919084,0.906059
